# CREMI Dataset
#### Team: Benedict Au, Amanda Zang, Jenny Zhen
#### Apr 1, 2020

In [1]:
import h5py
import os
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 1. CREMI Data Format Specification

Volumes and annotations are stored in a single HDF5 file with the following datasets:

#### Volumes
The volumes are stored in row-major format, i.e., with dimensions (depth,height,width). All volumes have an attribute resolution that specifies the size of voxels (depth,height,width) in nm. In the padded versions of the datasets, the raw volume provides much more context. In this case, the label volumes have an additional offset attribute (z,y,x) in nm that marks the beginning of the label volumes in the raw volume.

- `volumes/raw`: gray scale pixels
- `volumes/labels/clefts`: synaptic cleft segmentation
- `volumes/labels/neuron_ids`: neuron segmentation

#### Annotations
Annotations for pre- and post-synaptic partner pairs are provided as well. All locations are given as (z,y,x) in nm. In the padded versions of the datasets, the raw volume provides much more context. In this case, the annotation group has an additional offset attribute (z,y,x) in nm, which is the same as for the label volumes. All locations are relative to this offset, such that they are the same as in the unpadded version.

- `annotations/ids`: list of ids
- `annotations/locations`: list of annotation locations as (z,y,x) in nm, same order as ids
- `annotations/type`: list of annotation types (presynaptic_site or postsynaptic_site), same order as ids
- `presynaptic_site/partners`: list of id tuples (pre, post) linking pre- and postsynaptic partners
- `comments/target_ids`: list of ids for annotations that have a comment
- `comments/comments`: list of comments, same order as target_ids

### Viewing CREMI dataset hdf5 file hierarchy

In h5py, groups work like dictionaries, and datasets work like NumPy arrays. 

Here is the file hierarchy of Sample A:

In [2]:
filename = "./data_cremi/sample_A_20160501.hdf"
f = h5py.File(filename, 'r')

os.system("h5dump -n ./data_cremi/sample_A_20160501.hdf")
result = subprocess.run(["h5dump", "-n", filename], stdout=subprocess.PIPE)
print(result.stdout.decode('utf-8'))

HDF5 "./data_cremi/sample_A_20160501.hdf" {
FILE_CONTENTS {
 group      /
 group      /__DATA_TYPES__
 datatype   /__DATA_TYPES__/Enum_Boolean
 datatype   /__DATA_TYPES__/String_VariableLength
 group      /annotations
 group      /annotations/comments
 dataset    /annotations/comments/comments
 dataset    /annotations/comments/target_ids
 dataset    /annotations/ids
 dataset    /annotations/locations
 group      /annotations/presynaptic_site
 dataset    /annotations/presynaptic_site/partners
 dataset    /annotations/types
 group      /volumes
 group      /volumes/labels
 dataset    /volumes/labels/clefts
 dataset    /volumes/labels/neuron_ids
 dataset    /volumes/raw
 }
}



In greater detail:

In [3]:
result = subprocess.run(["h5dump", "-H", "-A", "0", filename], stdout=subprocess.PIPE)
print(result.stdout.decode('utf-8'))

HDF5 "./data_cremi/sample_A_20160501.hdf" {
GROUP "/" {
   GROUP "__DATA_TYPES__" {
      DATATYPE "Enum_Boolean" H5T_ENUM {
         H5T_STD_I8LE;
         "FALSE"            0;
         "TRUE"             1;
      };
      DATATYPE "String_VariableLength" H5T_STRING {
         STRSIZE H5T_VARIABLE;
         STRPAD H5T_STR_NULLTERM;
         CSET H5T_CSET_ASCII;
         CTYPE H5T_C_S1;
      };
   }
   GROUP "annotations" {
      GROUP "comments" {
         DATASET "comments" {
            DATATYPE  H5T_STRING {
               STRSIZE H5T_VARIABLE;
               STRPAD H5T_STR_NULLTERM;
               CSET H5T_CSET_UTF8;
               CTYPE H5T_C_S1;
            }
            DATASPACE  SIMPLE { ( 17 ) / ( 17 ) }
         }
         DATASET "target_ids" {
            DATATYPE  H5T_STD_U64LE
            DATASPACE  SIMPLE { ( 17 ) / ( 17 ) }
         }
      }
      DATASET "ids" {
         DATATYPE  H5T_STD_U64LE
         DATASPACE  SIMPLE { ( 432 ) / ( 432 ) }
      }
      DATASET

## 2. Volumes EDA: raw, clefts, neuron_ids datasets


### 2.1 Plot one slice

Visualise one slice of raw data, clefts labels and neuron_ids labels:

In [4]:
raw = f["/volumes/raw"]
clefts = f["/volumes/labels/clefts"]
neuron_ids = f["/volumes/labels/neuron_ids"]

img = (raw, clefts, neuron_ids)


In [ ]:
layer = 0

raw_layer = raw[layer]
clefts_layer = clefts[layer]
neuron_ids_layer = neuron_ids[layer]

print("Dataset dimensions: {}".format(raw.shape), "\n")
print("Layer {}: \n".format(layer))
print(raw_layer, "\n")

fig, axarr = plt.subplots(1,3,figsize=(15,15))

axarr[0].imshow(raw_layer)
axarr[0].title.set_text('Raw data')
axarr[1].imshow(clefts_layer)
axarr[1].title.set_text('Cleft labels')
axarr[2].imshow(neuron_ids_layer)
axarr[2].title.set_text('Neuron_id labels')

### 2.2 Plot all slices


In [ ]:
import ipywidgets as ipyw
from mpl_toolkits.axes_grid1 import make_axes_locatable

class CREMIImageViewer:
    """ Viewing CREMI volumetric image slices in Jupyter notebooks. 
    
    User can interactively select the slice plane and view the images 
    at each slice.
    
    Keyword arguments:
    img     -- 3-element tuple containing (raw, clefts, neuron_ids) data
    figsize -- to set the size of the figure (default (15,15))
    """
    
    def __init__(self, img, figsize=(18,18)):
        self.img = img
        self.figsize = figsize
        self.volume1 = img[0]
        self.volume2 = img[1]
        self.volume3 = img[2]
        self.v1 = [np.min(self.volume1), np.max(self.volume1)]
        self.v2 = [np.min(self.volume2), np.max(self.volume2)]
        self.v3 = [np.min(self.volume3), np.max(self.volume3)]
        
        """Call to select slice plane"""
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','y-z', 'z-x'], value='y-z', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        """Transpose the volume to orient according to the slice plane selection"""
        orient = {"x-y": [0,1,2], "y-z":[1,2,0], "z-x":[2,0,1]}
        self.vol1 = np.transpose(self.volume1, orient[view])
        self.vol2 = np.transpose(self.volume2, orient[view])
        self.vol3 = np.transpose(self.volume3, orient[view])
        maxZ = self.vol1.shape[2] - 1
        
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        """Plot slice for the given plane and slice"""
        self.fig, (self.ax1, self.ax2, self.ax3) = plt.subplots(1,3,figsize=self.figsize)
        
        self.divider1 = make_axes_locatable(self.ax1)
        self.cax1 = self.divider1.append_axes('right', size='5%', pad=0.05)
        self.im1 = self.ax1.imshow(self.vol1[:,:,z], vmin=self.v1[0], vmax=self.v1[1], cmap="Greys")
        self.fig.colorbar(self.im1, cax=self.cax1, orientation='vertical')
        self.ax1.title.set_text('Raw data')
        
        self.divider2 = make_axes_locatable(self.ax2)
        self.cax2 = self.divider2.append_axes('right', size='5%', pad=0.05)
        self.im2 = self.ax2.imshow(self.vol2[:,:,z], vmin=self.v2[0], vmax=self.v2[1], cmap="binary")
        self.fig.colorbar(self.im2, cax=self.cax2, orientation='vertical')
        self.ax2.title.set_text('Cleft labels')

        self.divider3 = make_axes_locatable(self.ax3)
        self.cax3 = self.divider3.append_axes('right', size='5%', pad=0.05)
        self.im3 = self.ax3.imshow(self.vol3[:,:,z], vmin=self.v3[0], vmax=self.v3[1])
        self.fig.colorbar(self.im3, cax=self.cax3, orientation='vertical')
        self.ax3.title.set_text('Neuron_id labels');
        
        plt.show();
 

In [ ]:
CREMIImageViewer(img)

### 2.3 Plot single neuron in all slices 


In [5]:
import ipywidgets as ipyw
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap

class CREMINeuronViewer:
    """ Viewing CREMI volumetric image slices in Jupyter notebooks. 
    
    User can interactively select both the neuron_id to be mapped and 
    the slice plane, then view the images at each slice. The neurons 
    and their ids are sorted according to size in decreasing order.
    
    Keyword arguments:
    img     -- 3-element tuple containing (raw, clefts, neuron_ids) data
    figsize -- to set the size of the figure (default (15,15))
    """
    
    def __init__(self, img, figsize=(25,25)):
        self.img = img
        self.figsize = figsize
        self.volume1 = img[0]
        self.volume2 = img[1]
        self.volume3 = img[2]
        
        self.volume3_flat = self.volume3[:].flatten()
        self.unique_ids, self.counts = np.unique(self.volume3_flat, return_counts=True) 
        self.unique_id_count = np.column_stack([self.unique_ids, self.counts])
        self.unique_id_count = self.unique_id_count[self.unique_id_count[:,1].argsort()[::-1]].astype(int)
        self.sorted_id = self.unique_id_count[:,0].tolist()
        self.sorted_count = self.unique_id_count[:,1].tolist()

        """Create custom color map with increasing alpha for synapse overlay"""
        cmap1 = pl.cm.autumn
        self.my_cmap1 = cmap1(np.arange(cmap1.N))
        self.my_cmap1[:,-1] = np.linspace(1, 0, cmap1.N)
        self.my_cmap1 = ListedColormap(self.my_cmap1)    
        
        """Create custom color map with increasing alpha for neuron overlay"""
        cmap2 = pl.cm.winter_r
        self.my_cmap2 = cmap2(np.arange(cmap2.N))
        self.my_cmap2[:,-1] = np.linspace(0, 1, cmap2.N)
        self.my_cmap2 = ListedColormap(self.my_cmap2)
    
        """Call to select slice plane"""
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','y-z', 'z-x'], 
            value='y-z', 
            description='Slice plane selection:', 
            disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        """Transpose the volume to orient according to the slice plane selection"""
        orient = {"x-y": [0,1,2], "y-z":[1,2,0], "z-x":[2,0,1]}
        self.vol1 = np.transpose(self.volume1, orient[view])
        self.vol2 = np.transpose(self.volume2, orient[view])
        self.vol3 = np.transpose(self.volume3, orient[view])
        self.maxZ = self.vol1.shape[2] - 1
        
        """Call to select neuron_id"""
        ipyw.interact(self.neuron_selection, neuron=ipyw.SelectionSlider(
            options=self.sorted_id,
            value=self.sorted_id[0],
            description='Neuron_id:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True))
        
    
    def neuron_selection(self, neuron):
        """Transform neuron_ids such that selected neuron is 1 else 0"""
        self.neuron = neuron
        self.vol3r = (self.vol3==self.neuron).astype(int)

        """Call to view a slice within the selected slice plane"""
        ipyw.interact(self.plot_slice, z=ipyw.IntSlider(
            min=0, 
            max=self.maxZ, 
            step=1, 
            continuous_update=False, 
            description='Image Slice:'))      
    
    def plot_slice(self, z):
        """Plot slice for the given plane and slice"""
        self.fig, (self.ax1, self.ax2, self.ax3) = plt.subplots(1,3,figsize=self.figsize)
        
        self.divider1 = make_axes_locatable(self.ax1)
        self.cax1 = self.divider1.append_axes('right', size='5%', pad=0.05)
        self.im1 = self.ax1.imshow(self.vol1[:,:,z], cmap="Greys")
        self.fig.colorbar(self.im1, cax=self.cax1, orientation='vertical')
        self.ax1.title.set_text('Raw data')
        self.im1 = self.ax1.imshow(self.vol2[:,:,z], cmap=self.my_cmap1) # vol2 overlay
        self.im1 = self.ax1.imshow(self.vol3r[:,:,z], cmap=self.my_cmap2) # vol3r overlay
        
        
        self.divider2 = make_axes_locatable(self.ax2)
        self.cax2 = self.divider2.append_axes('right', size='5%', pad=0.05)
        self.im2 = self.ax2.imshow(self.vol2[:,:,z], cmap=self.my_cmap1)
        self.fig.colorbar(self.im2, cax=self.cax2, orientation='vertical')
        self.ax2.title.set_text('Clefts')

        self.divider3 = make_axes_locatable(self.ax3)
        self.cax3 = self.divider3.append_axes('right', size='5%', pad=0.05)
        self.im3 = self.ax3.imshow(self.vol3r[:,:,z], cmap=self.my_cmap2) # vol3r
        self.fig.colorbar(self.im3, cax=self.cax3, orientation='vertical')
        self.ax3.title.set_text('Neuron {}'.format(self.neuron));
        
        plt.show();
 

In [ ]:
### 2.4 Distributions

In [ ]:
from scipy import stats

datasets = [raw, clefts, neuron_ids]
dataset_names = ["Raw", "Clefts", "Neuron_ids"]

for dataset, dataset_name in zip(datasets, dataset_names):
    print("Dataset: {}".format(dataset_name))
    print("Shape: {}".format(dataset.shape))
    dataset_flat = dataset[:].flatten()
    print("Number of pixels: {:,}".format(len(dataset_flat)))
    
    # Descriptive stats
    des = stats.describe(dataset_flat)
    print("Min: {:,}".format(des.minmax[0]))
    print("Max: {:,}".format(des.minmax[1]))
    print("Mean: {0:1.3e}".format(des.mean))
    print("Variance: {0:1.3e}".format(des.variance))
    print("Skewness: {0:1.3e}".format(des.skewness))
    print("kurtosis: {0:1.3e}".format(des.kurtosis))
    print("Unique values: {}\n".format(len(np.unique(dataset_flat))))
    
    # Sampling to visualise
    dataset_sample = dataset[0] # take first slice as sample 
    # dataset_sample = np.random.choice(
    #     dataset_flat, int(0.01*len(dataset_flat))) # take 1% of vol as sample
    f, ax = plt.subplots(figsize=(6, 4))
    ax = sns.distplot(dataset_sample, axlabel="{} sample".format(dataset_name), kde=False, rug=False)
    ax.set_title('Histogram: {} sample'.format(dataset_name))
    ax.set_ylabel('Freqency')
    plt.show();
    
    print("\n")

In [6]:
CREMINeuronViewer(img)

interactive(children=(RadioButtons(description='Slice plane selection:', index=1, options=('x-y', 'y-z', 'z-x'…